### create xml 

In [1]:
import xml.etree.ElementTree as ET

In [2]:
root = ET.Element ("centerlines")
root.set('version','1.0')

single_node  = ET.SubElement(root, 'nodes')
links        = ET.SubElement(root, 'links')

### read centerline  (id, length ) .csv

In [3]:
import csv

In [4]:
with open ("centerline_points.csv",'rt',encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader, None)
    current_group = None
    current_link = None 
    for row in reader :
        WKT, FID, length,vertex_ind,vertex_par,vertex_p_1,distance,angle,nodeId = row 
        #single_node
        ET.SubElement(single_node, 'node',{'nodeId': nodeId, 'geometry':WKT[7:-1], 'linkId':FID, 'vertex_ind':vertex_ind})
        #links
        if current_group is None or FID != current_group.get('linkId'):
            current_group = ET.SubElement(links, 'link',{'linkId':FID,'length':length })
        
        
        if (vertex_ind=='0'):
            current_link = ET.SubElement(current_group, 'nodes',{'nodeId':nodeId})   
        else: 
            list_nodeId = current_link.get('nodeId')
            current_link.set('nodeId',list_nodeId+" "+nodeId)


In [5]:
#set node1 node2 of link
for link in links.findall('link'):
    list_node = link.find('nodes').get('nodeId').split(' ')
    link.set('node1', list_node[0])
    link.set('node2',list_node[-1])

In [6]:
tree = ET.ElementTree(root)
ET.indent(tree)
tree.write('output.xml',encoding='utf-8')